In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
import glob
import os
import contextily as ctx
from scipy.spatial import cKDTree
from shapely.geometry import Point
import json
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map, thread_map
pd.set_option('min_rows', 30)
import sys
sys.path.append('..')
from importlib import reload
# import src.utils as utils
# reload(utils)
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12, 12)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
max_workers = 30

In [ ]:
def gdf2tree(gdf):
    return cKDTree(np.array(list(gdf[~gdf.geometry.isna()].geometry.apply(lambda x: (x.x, x.y)))))

In [ ]:
%%time
## read in
df = pd.read_csv("restricted/BCs_issued_by_AUP_TLADCs_2021FEB (1).csv", encoding='cp1252')
bcs = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.X_Coordinate, df.Y_Coordinate))

In [ ]:
ax = bcs.sample(1000).plot(column='Building_Type_Group', legend=True, figsize=(20,20))
ctx.add_basemap(ax, crs=bcs.crs)

In [ ]:
# addressing of building consents is very inconsistent
# some building consents have street name in ADDRESS_2, some have suburb, some even have 'Auckland'
bcs[bcs.ADDRESS_2 == 'Auckland'].sample(5)

In [ ]:
# one address has no ADDRESS_1, but an ADDRESS_2 with a leading digit
display(bcs[bcs.ADDRESS_1.isna() & ~bcs.ADDRESS_2.isna()][bcs[bcs.ADDRESS_1.isna() & ~bcs.ADDRESS_2.isna()].ADDRESS_2.str.contains('^[0-9]', regex=True)])
# move address 2 to address 1 for that one BC...
bcs.loc[bcs.OBS == 146813, 'ADDRESS_1'] = "25 HOLLYFORD DRV"
bcs.loc[bcs.OBS == 146813, 'ADDRESS_2'] = ""

In [ ]:
%%time
# get number and name of street (but not 'road', 'street', 'place' etc)
# this can be used to match addresses with building consents

def number_name_bc(x):
    """extract street number and first complete word of the street name from building consents"""
    if x.ADDRESS_1 is None:
        pass
    else:
        # get number and first word of address
        joined_address = ' '.join([str(x[f'ADDRESS_{i}']) for i in [1,2, 3]]).lower()
        return ' '.join(joined_address.split(' ')[:2])

def full_address_bc(x):
    """extract full address from building consents"""
    if x.ADDRESS_1 is None:
        pass
    else:
        # get number and first word of address
        joined_address = ' '.join([str(x[f'ADDRESS_{i}']) for i in [1,2, 3] if not str(x[f'ADDRESS_{i}']) == 'nan']).lower()
        return joined_address

bcs['number_name'] = bcs.apply(number_name_bc, axis=1)
bcs['full_address'] = bcs.apply(full_address_bc, axis=1)

# there will still be some cases where there is no street number
display(bcs.number_name.sample(5))

In [ ]:
# check the number of rows where number_name has its last character as numeric; this should be zero
print(sum(bcs['number_name'].apply(lambda x: x[-1] in [i for i in range(10)])))

#### Split into three branches:  
- those with an address number range (e.g. 10-20)
- those with a single number (e.g. 10)
- those with no number  

Those with no number cannot be matched with addresses to get a more accuract coordinate.

In [ ]:
bcs_non_na = bcs[~bcs.ADDRESS_1.isna()]
bcs_ranged = bcs_non_na[bcs_non_na.ADDRESS_1.str.contains('^[0-9]+-[0-9]', regex=True)]
# get all bcs which start with a digit
bcs_numbered = bcs_non_na[bcs_non_na.ADDRESS_1.str.contains('^[0-9]', regex=True)]
# exclude those in bcs_ranged
bcs_numbered = bcs_numbered[~bcs_numbered.ADDRESS_1.str.contains('^[0-9]+-[0-9]', regex=True)]
bcs_others = pd.concat([bcs_non_na[~bcs_non_na.ADDRESS_1.str.contains('^[0-9]', regex=True)], bcs[bcs.ADDRESS_1.isna()]])
print('non na bcs:', len(bcs_non_na))
print()
print('1. ranged:', len(bcs_ranged))
print('2. numbered:', len(bcs_numbered))
print('3. non ranged/non numbered:', len(bcs_others))
print('total bcs:', len(bcs))
print('sum of 1, 2, 3:', len(bcs_ranged) + len(bcs_numbered) + len(bcs_others))

## How many building consents have coordinates that match with the correct parcel?  
Building consents may have geocoordinates that are incorrect. We can quantify this by matching building consents to parcels, matching addresses to parcels, and then see whether or not building consents and their corresponding addresses are matched to the same parcels.  
(BUT sometimes the geocoordinates could be correct, and the address wrong! E.g. mispelt. Match on the number and first word of address to mitigate this.)  
If a building consent matches in this part, then we use the geocoordinate, and LINZ_MATCH_CODE = 1.  
Currently this only does numbered building consents.

In [ ]:
%%time
## read in address dataset and add number_name, just like for BCs
# https://data.linz.govt.nz/layer/53353-nz-street-address/
addresses = gpd.read_file('input/lds-nz-street-address-GPKG-CLIPPED.gpkg').to_crs(2193)
def number_name_addresses(x):
    return ' '.join(x.full_address.split(' ')[:2]).lower()
addresses['number_name'] = addresses.apply(number_name_addresses, axis=1)

In [ ]:
%%time
parcels = gpd.read_file('input/NZ_Primary_Parcels_Nov_2016_filtered.gpkg').to_crs(2193)

In [ ]:
%%time
parcels_addressed = gpd.sjoin(parcels, addresses)
parcels_addressed = parcels_addressed.rename(columns={'index_right': 'address_index'})

In [ ]:
print('addresses match uniquely to parcels')
print(np.unique(parcels_addressed.address_index.value_counts(), return_counts=True))
print()
print("but parcels don't match uniquely to addresses")
print(np.unique(parcels_addressed.index.value_counts(), return_counts=True))

In [ ]:
%%time
parcels_addressed_bced = gpd.sjoin(parcels_addressed, bcs_numbered)
parcels_addressed_bced = parcels_addressed_bced.rename(columns={'index_right': 'bc_index', 'number_name_right': 'bc_number_name', 'number_name_left': 'address_number_name'})

In [ ]:
%%time
matches = []
for i in tqdm(bcs_numbered.index):
    matched_address_number_name = parcels_addressed_bced[parcels_addressed_bced.bc_index == i].address_number_name.tolist()
    matches.append(bcs_numbered.loc[i].number_name in matched_address_number_name)
print('proportion of building consents that match to a parcel with the right address information')
sum(matches) / len(matches)

In [ ]:
bcs_numbered['LINZ_MATCH_CODE'] = [1 if m else None for m in matches]

## Get more accurate coordinates from NZ addresses dataset  
Check addresses within radius r for those that match on number and first word of address.

In [ ]:
%%time
addresses_tree = gdf2tree(addresses)
print('addresses tree created')
bcs_numbered_tree = gdf2tree(bcs_numbered)
print('bcs numbered tree created')
bcs_ranged_tree = gdf2tree(bcs_ranged)
print('bcs ranged tree created')

### Finding r  
Find the best distance threshold  
(this was done on sample from all numbered bcs. Perhaps results would differ slightly if limiting to those that did not already match to the correct parcel.)

In [ ]:
# test some radii to see which is suitable
bcs_numbered_sample = bcs_numbered.sample(500)
bcs_numbered_sample_tree = cKDTree(np.array(list(bcs_numbered_sample.geometry.apply(lambda x: (x.x, x.y)))))
matches = {}
for r in [10, 50, 100, 175] + list(range(250, 3251, 250)):
for r in test_thresholds:
    matches[r] = []
    # list of lists: ith sub list contains indices of use_tree points within r of the ith data_tree point
    bcs_numbered_neighbours = bcs_numbered_sample_tree.query_ball_tree(addresses_tree, r)
    for i, neighbours in tqdm(enumerate(bcs_numbered_neighbours)):
        # check how many matches there are
        matches[r].append(np.sum(bcs_numbered_sample.iloc[i].number_name == addresses.iloc[neighbours].number_name))

In [ ]:
# for r in sorted(matches.keys()):
#     print('####', r, '####')
#     print('match:', np.sum(np.array(matches[r]) == 1) / len(matches[r]))
#     print('ambiguous:', np.sum(np.array(matches[r]) > 1) / len(matches[r]))
#     print('no match:', np.sum(np.array(matches[r]) == 0) / len(matches[r]))
r_list = list(sorted(matches.keys()))
ax = plt.subplot(1,1,1)
ax.plot(r_list, [np.sum(np.array(matches[r]) == 1) / len(matches[r]) for r in r_list], label='match')
ax.plot(r_list, [np.sum(np.array(matches[r]) > 1) / len(matches[r]) for r in r_list], label='ambiguous')
ax.plot(r_list, [np.sum(np.array(matches[r]) == 0) / len(matches[r]) for r in r_list], label='no match')
ax.legend()
plt.xlabel('radius')
plt.ylabel('proportion')
plt.grid()

### perform matching

In [ ]:
# add empty rows with indices -1 and -2; these will be retrieved if there is no match address match
# this enables us to use a list of indices to get all information in one step, rather than iterating over in a for loop
addresses.loc[-1] = addresses.loc[1]
for c in addresses.columns:
    addresses.loc[-1, c] = np.nan
    addresses.loc[-1, c]
addresses.loc[-2] = addresses.loc[-1]

#### Non ranged addresses  
Matches with more than 80% success rate for numbered bcs.  
Matches more than half the time for numbered bcs that weren't already on the correct parcel.

In [ ]:
%%time
r = 1250
bcs_to_match = bcs_numbered[bcs_numbered.LINZ_MATCH_CODE.isna()]
bc_tree = cKDTree(np.array(list(bcs_to_match.geometry.apply(lambda x: (x.x, x.y)))))
# this creates a list of lists, where the ith list contains the the neighbours of the ith bc
bc_tree_neighbours = bc_tree.query_ball_tree(addresses_tree, r)

def perform_matching(item):
    """check if there is a neighbouring point that matches on number_name
    
    arguments:
    item - (i, neighbours), where neighbours is the result of query_ball_tree
    
    returns:
        - the index of the matching address, if there is a unique match
        - -1 if there is more than one match
        - -2 if there are no matches
    """
    i, neighbours = item
    # subset right to those within r distance
    right_neighbours = addresses.iloc[neighbours]
    # check how many matches there are
    match_indicator = bcs_to_match.iloc[i]['number_name'] == right_neighbours['number_name']
    if np.sum(match_indicator) == 1:
        # extract the index from right_neighbours
        # extracting the index only and later subsetting the full addresses dataframe
        # results in a ~4x speed up over extracting the requisite addresses information within this for loop
        return right_neighbours[match_indicator].index[0]
    elif np.sum(match_indicator) > 1:
        return -1
    else:
        return -2

match_indices = process_map(perform_matching, enumerate(bc_tree_neighbours), max_workers=max_workers, chunksize=100, total=len(bc_tree_neighbours))
print('#### r =', r, '####')
print('match:', np.sum(np.array(match_indices) > -1) / len(match_indices))
print('ambiguous:', np.sum(np.array(match_indices) == -1) / len(match_indices))
print('no match:', np.sum(np.array(match_indices) == -2) / len(match_indices))

In [ ]:
bcs_to_match['addresses_geometry'] = addresses.loc[match_indices].geometry.reset_index(drop=True).tolist()
bcs_to_match['addresses_full_address'] = addresses.loc[match_indices].full_address.reset_index(drop=True).tolist()
bcs_to_match['euc_distance'] = [np.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2) for a, b in zip(bcs_to_match.geometry.apply(lambda x: (x.x, x.y)), bcs_to_match['addresses_geometry'].apply(lambda x: (x.x, x.y) if not pd.isna(x) else (0, 0)))]

In [ ]:
bcs_to_match[~bcs_to_match.addresses_full_address.isna()].sort_values('euc_distance').reset_index(drop=True).euc_distance.plot()
plt.xlabel('Building Consent')
plt.ylabel('Distance to Matched Address (metres)')

In [ ]:
bcs_to_match.sample(20).sort_values('euc_distance')[['full_address', 'addresses_full_address', 'euc_distance']].to_csv('address_match_sample.csv')

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(bcs_to_match.sample(20).sort_values('euc_distance')[['full_address', 'addresses_full_address', 'euc_distance']])

### Ranged addresses

In [ ]:
# all number ranges that have a non digit in them
for index, row in tqdm(bcs_ranged.iterrows(), total=len(bcs_ranged)):
    r = row.number_name.split(' ')[0].split('/')[0]
    if not all([s.isdecimal() for s in r.split('-')]):
        print(row.number_name.split(' ')[0])

In [ ]:
# check the proportion of ranged addresses that have endpoints matching in being either even or odd
# if this proportion is high, then 20-26 should become 20, 22, 24, 26
endpoints_match = []
for index, row in tqdm(bcs_ranged.iterrows(), total=len(bcs_ranged)):
    r = row.number_name.split(' ')[0].split('/')[0]
    if all([s.isdecimal() for s in r.split('-')]):
        endpoints = [s for s in r.split('-')]
        if '/' not in row.number_name:
            endpoints_match.append(int(endpoints[0]) % 2) == (int(endpoints[1]) % 2)
# proportion is around what you would expect by chance
sum(endpoints_match) / len(endpoints_match)

In [ ]:
def range_expand(r):
    """given a range r, like '20-23', expand the range on one side of the road, e.g. ['20', '21', '22', '23']"""
    if '/' in r:
        r, suffix = r.split('/')
        suffix = '/' + suffix
    else:
        suffix = ''
    r1, r2 = r.split('-')
    if r1.isdecimal() and r2.isdecimal():
        return [str(i) + suffix for i in range(int(r1), int(r2) + 1)]
    else:
        return r + suffix

In [ ]:
range_expand('20-28')

In [ ]:
range_expand('1-5/32a')

In [ ]:
# get a list of gdfs, one gdf per ranged bc 
# each gdf expands a row of bcs_ranged into many rows, one for each number in the range
expanded_gdfs = []
for index, row in tqdm(bcs_ranged.iterrows(), total=len(bcs_ranged)):
    number_name = row.number_name.split(' ')
    number = number_name[0]
    name = number_name[1]
    range_expanded = range_expand(number)
    
    range_rows = [row.copy() for _ in range(len(range_expanded))]
    for i, rr in enumerate(range_rows):
        rr.number_name = range_expanded[i] + ' ' + name
    
    expanded_gdfs.append(pd.DataFrame(range_rows))
expanded_gdfs[0]